In [3]:
import pandas as pd
import numpy as np

factor_prices = pd.DataFrame(
    {
        "Factor 1": [100, 105, 110, 115, 120],
        "Factor 2": [200, 205, 210, 215, 220],
        "Factor 3": [300, 305, 310, 315, 320],
    },
    index=range(5),
)
factor_prices.head()

,Factor 1,Factor 2,Factor 3
0,100,200,300
1,105,205,305
2,110,210,310
3,115,215,315
4,120,220,320


In [5]:
log_returns = np.log(factor_prices / factor_prices.shift(1))
print(log_returns.dropna()) # type: ignore

   Factor 1  Factor 2  Factor 3
1  0.048790  0.024693  0.016529
2  0.046520  0.024098  0.016261
3  0.044452  0.023530  0.016000
4  0.042560  0.022990  0.015748
